## 1. Importing necessary libraries

In [1]:
import sagemaker
import boto3 #access s3 bucket

from sagemaker.amazon.amazon_estimator import image_uris
from sagemaker.session import s3_input, Session

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


## 2. Create S3 bucket

In [2]:
bucket_name = 'cora-bank-application-ex' # the name has to be globally unique
my_region = boto3.session.Session().region_name # set the region of the instance
print(my_region)

us-east-2


In [3]:
s3 = boto3.resource('s3')
try:
    if my_region == 'us-east-2':
        s3.create_bucket(Bucket=bucket_name,
                         CreateBucketConfiguration={'LocationConstraint': my_region}) # adding this b/c my region is not the default region `us-east-1`
       
    print('s3 bucket created successfully')
except Exception as e:
    print('s3 error: ',e)

s3 error:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


###  --> set an output path where the trained model will be saved

In [4]:
prefix = 'xgboost-as-a-built-in-algo'
output_path = 's3://{}/{}/output'.format(bucket_name,prefix)
print(output_path)
# by setting this, every model being trained based on time will be stored in proper folders in this path

s3://cora-bank-application-ex/xgboost-as-a-built-in-algo/output


## 3. Downloading the dataset and storing in s3

In [5]:
import pandas as pd
import urllib
try: 
    urllib.request.urlretrieve("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv')
except Exception as e:
    print('Data load error: ' ,e)
try:
    model_data = pd.read_csv('bank_clean.csv', index_col = 0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ', e)


Success: downloaded bank_clean.csv
Success: Data loaded into dataframe.


### --> Train Test Split

In [6]:
import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))]) # 70% as train 
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


### --> Saving Train and Test into buckets

In [7]:
import os
# put the dependent column to the front
pd.concat([train_data['y_yes'],train_data.drop(['y_no','y_yes'], axis=1)],axis=1).to_csv('train.csv', index=False, header=False)
    # axis=1 means column wise operation rather than row-wise
    #index=False, no indexes for rows 
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')
    #  telling SageMaker: Where the training data is in S3 & What format the data is in

In [8]:
pd.concat([test_data['y_yes'],test_data.drop(['y_no','y_yes'], axis=1)],axis=1).to_csv('test.csv', index=False, header=False)
    
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

## Build Model XGBoost - Inbuilt Algorithm

In [9]:
container= image_uris.retrieve(framework='xgboost',
    region=boto3.Session().region_name,
    version='1.0-1')
# this line automatically looks for XGBoost image URI and builds an XGBoost container.
# SageMaker provides pre-built images(blueprint/a packaged env) for common algorithms like XGBoost, Linear Learner, etc


In [10]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }
     

In [11]:
# construct a SageMaker estimator that calls the xgboost-container
from sagemaker.estimator import Estimator
from sagemaker import get_execution_role
estimator = Estimator(
    image_uri=container,
    role=get_execution_role(),
    instance_count=1,
    instance_type='ml.m5.2xlarge',
    volume_size=5,  # in GB
    max_run=300,
    use_spot_instances=True,
    max_wait=600,
    output_path=output_path,
    hyperparameters=hyperparameters
)

In [12]:
estimator.fit({'train':s3_input_train, 'validation':s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2025-07-02-21-38-03-832


2025-07-02 21:38:04 Starting - Starting the training job...
2025-07-02 21:38:35 Downloading - Downloading input data...
2025-07-02 21:38:51 Downloading - Downloading the training image...
2025-07-02 21:39:31 Training - Training image download completed. Training in progress...[2025-07-02 21:39:47.963 ip-10-0-236-231.us-east-2.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Single node trainin

## Deploy the Ml Model + Prediction

In [13]:
xgb_predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2025-07-02-21-53-52-014
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2025-07-02-21-53-52-014
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2025-07-02-21-53-52-014


------!

In [18]:
from sagemaker.serializers import CSVSerializer
test_data_array = test_data.drop(['y_no','y_yes'], axis=1).values

xgb_predictor.content_type = 'text/csv'  # Tells SageMaker to expect input as CSV-formatted text.
xgb_predictor.serializer = CSVSerializer()

predictions= xgb_predictor.predict(test_data_array).decode('utf-8')    #str output
predictions_array= np.fromstring(predictions[1:], sep=',') # turn the prediction into an array
# in some cases, it might contain a leading newline (\n) or extra character, like:'\n0.1,0.9,0.2,0.8'
print(predictions_array.shape)

(12357,)


### --> Confusion Matrix

In [19]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))
     


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 



## Delete the Endpoint

In [23]:
# sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)

session = Session()
try:
    session.delete_endpoint(endpoint_name=xgb_predictor.endpoint_name)
    print(f"Deleted endpoint: {xgb_predictor.endpoint_name}")
except Exception as e:
    print(f"Could not delete endpoint: {e}")
    
bucket_to_delete= boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2025-07-02-21-53-52-014


Could not delete endpoint: An error occurred (ValidationException) when calling the DeleteEndpoint operation: Could not find endpoint "sagemaker-xgboost-2025-07-02-21-53-52-014".


[{'ResponseMetadata': {'RequestId': '3E46D9HYH55F3Z10',
   'HostId': 'KpLGFFXhCol58IcBsEp3zlkljnrwVJ7Yd3Kqv8WWUKiuDYITGiNboJu7rzwtn3gie3XWMKgASto=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'KpLGFFXhCol58IcBsEp3zlkljnrwVJ7Yd3Kqv8WWUKiuDYITGiNboJu7rzwtn3gie3XWMKgASto=',
    'x-amz-request-id': '3E46D9HYH55F3Z10',
    'date': 'Wed, 02 Jul 2025 22:29:23 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2025-07-02-21-38-03-832/debug-output/claim.smd'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2025-07-02-21-38-03-832/debug-output/index/000000000/000000000010_worker_0.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2025-07-02-21-38-03-832/profiler-output/system/training_job_end.ts'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-